In [62]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [63]:
# Connect to a pre-existing cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 days 1 hour 36 mins
H2O cluster version:,3.10.5.4
H2O cluster version age:,13 days
H2O cluster name:,sneha
H2O cluster total nodes:,1
H2O cluster free memory:,1.586 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [64]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.

df = h2o.import_file(path=_locate("prostate.csv"))

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [65]:
df.describe()

Rows:380
Cols:9




,ID,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON
type,int,int,int,int,int,int,real,real,int
mins,1.0,0.0,43.0,0.0,1.0,1.0,0.3,0.0,0.0
mean,190.5,0.4026315789473684,66.03947368421049,1.0868421052631572,2.2710526315789488,1.1078947368421048,15.408631578947375,15.812921052631573,6.3842105263157904
maxs,380.0,1.0,79.0,2.0,4.0,2.0,139.70000000000002,97.60000000000001,9.0
sigma,109.84079387914127,0.4910743389630552,6.527071269173311,0.3087732580252793,1.0001076181502861,0.3106564493514939,19.99757266856046,18.347619967271175,1.0919533744261092
zeros,0,227,0,3,0,0,0,167,2
missing,0,0,0,0,0,0,0,0,0
0,1.0,0.0,65.0,1.0,2.0,1.0,1.4000000000000001,0.0,6.0
1,2.0,0.0,72.0,1.0,3.0,2.0,6.7,0.0,7.0
2,3.0,0.0,70.0,1.0,1.0,2.0,4.9,0.0,6.0


In [66]:

# Remove ID from training frame
train = df.drop("ID")

In [67]:
# For VOL & GLEASON, a zero really means "missing"
vol = train['VOL']
vol[vol == 0] = None
train['VOL']=vol
gle = train['GLEASON']
gle[gle == 0] = None
train['GLEASON']=gle

In [68]:
# Convert CAPSULE to a logical factor
train['CAPSULE'] = train['CAPSULE'].asfactor()

In [69]:

# See that the data is ready
train.describe()

Rows:380
Cols:8




,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON
type,enum,int,int,int,int,real,real,int
mins,,43.0,0.0,1.0,1.0,0.3,1.92,4.0
mean,,66.03947368421049,1.0868421052631572,2.2710526315789488,1.1078947368421048,15.408631578947375,28.21084507042253,6.41798941798942
maxs,,79.0,2.0,4.0,2.0,139.70000000000002,97.60000000000001,9.0
sigma,,6.527071269173311,0.3087732580252793,1.0001076181502861,0.3106564493514939,19.99757266856046,15.824317840954281,0.9906160127760705
zeros,,0,3,0,0,0,0,0
missing,0,0,0,0,0,0,167,2
0,0,65.0,1.0,2.0,1.0,1.4000000000000001,nan,6.0
1,0,72.0,1.0,3.0,2.0,6.7,nan,7.0
2,0,70.0,1.0,1.0,2.0,4.9,nan,6.0


In [70]:
# Run GBM
my_gbm = H2OGradientBoostingEstimator(distribution = "bernoulli", ntrees=50, learn_rate=0.1)

my_gbm.train(x=list(range(1,train.ncol)), y="CAPSULE", training_frame=train, validation_frame=train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [71]:
my_gbm_metrics = my_gbm.model_performance(train)
my_gbm_metrics.show()


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.07386057681148359
RMSE: 0.2717730244367229
LogLoss: 0.2678615063151671
Mean Per-Class Error: 0.07225533385160232
AUC: 0.979643546111543
Gini: 0.9592870922230861
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4552296658249728: 


,0,1,Error,Rate
0,212.0,15.0,0.0661,(15.0/227.0)
1,12.0,141.0,0.0784,(12.0/153.0)
Total,224.0,156.0,0.0711,(27.0/380.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4552297,0.9126214,155.0
max f2,0.3035822,0.9344059,195.0
max f0point5,0.5563743,0.9417040,128.0
max accuracy,0.4823319,0.9289474,147.0
max precision,0.9799473,1.0,0.0
max recall,0.1901482,1.0,231.0
max specificity,0.9799473,1.0,0.0
max absolute_mcc,0.4552297,0.8528780,155.0
max min_per_class_accuracy,0.4552297,0.9215686,155.0
max mean_per_class_accuracy,0.4552297,0.9277447,155.0


Gains/Lift Table: Avg response rate: 40.26 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0105263,0.9698129,2.4836601,2.4836601,1.0,1.0,0.0261438,0.0261438,148.3660131,148.3660131
,2,0.0210526,0.9640032,2.4836601,2.4836601,1.0,1.0,0.0261438,0.0522876,148.3660131,148.3660131
,3,0.0315789,0.9575014,2.4836601,2.4836601,1.0,1.0,0.0261438,0.0784314,148.3660131,148.3660131
,4,0.0421053,0.9503944,2.4836601,2.4836601,1.0,1.0,0.0261438,0.1045752,148.3660131,148.3660131
,5,0.05,0.9440268,2.4836601,2.4836601,1.0,1.0,0.0196078,0.1241830,148.3660131,148.3660131
,6,0.1,0.8904445,2.4836601,2.4836601,1.0,1.0,0.1241830,0.2483660,148.3660131,148.3660131
,7,0.15,0.8321802,2.4836601,2.4836601,1.0,1.0,0.1241830,0.3725490,148.3660131,148.3660131
,8,0.2,0.7822521,2.4836601,2.4836601,1.0,1.0,0.1241830,0.4967320,148.3660131,148.3660131
,9,0.3,0.6275345,2.2875817,2.4183007,0.9210526,0.9736842,0.2287582,0.7254902,128.7581699,141.8300654
,10,0.4,0.4626531,1.7647059,2.2549020,0.7105263,0.9078947,0.1764706,0.9019608,76.4705882,125.4901961


In [83]:
# # import h2o
# # h2o.init()
# # path = h2o.system_file("prostate.csv")
# h2o_df = train
# h2o_df.describe()




In [84]:
# from h2o.estimators.glm import H2OGeneralizedLinearEstimator
# help(H2OGeneralizedLinearEstimator)

In [85]:
# glm_multi_v1 = H2OGeneralizedLinearEstimator(
#                     model_id='glm_v1',            #allows us to easily locate this model in Flow
#                     family='multinomial',
#                     solver='L_BFGS')

In [86]:
# train, valid, test = train.split_frame([0.7, 0.15], seed=1234)
# print (train.describe())

In [87]:
# xvar = h2o_df[["AGE", "RACE", "PSA", "GLEASON"]]    #last column is Cover_Type, our desired response variable 
# yvar = h2o_df[["CAPSULE"]]    
# # gaussian_fit = h2o.glm(y = "VOL", x = ["AGE", "RACE", "PSA", "GLEASON"],
# # training_frame = h2o_df, family = "gaussian")


In [88]:
# glm_multi_v1.train(xvar, yvar, training_frame=train, validation_frame=valid)
# glm_multi_v1_metrics = glm_multi_v1.model_performance(train)
# glm_multi_v1_metrics.show()

In [89]:
# from h2o.estimators.random_forest import H2ORandomForestEstimator
# from h2o.estimators.gbm import H2OGradientBoostingEstimator

# from __future__ import print_function

In [90]:
# my_rf = H2ORandomForestEstimator(ntrees=50,
#                                  nfolds=5,
#                                  fold_assignment="Modulo",
#                                  keep_cross_validation_predictions=True,
#                                  seed=1)
# my_rf.train(x=xvar, y=yvar, training_frame=train)

In [91]:
# perf_rf_test = my_rf.model_performance(test)



In [92]:
# baselearner_best_auc_test = max(my_gbm_metrics.auc(), perf_rf_test.auc())

# print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))